In [2]:


# I copyied this from the setup file cause 
# it wasnt working too well without it
# directly in the notebook, so i just _made_ it part of
# the notebook


import matplotlib.pyplot as plt
import html 
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
# suppress warnings
import warnings;
warnings.filterwarnings('ignore');

# common imports
import pandas as pd
import numpy as np
import math
import re
import glob
import os
import sys
import json
import random
import pprint as pp
import textwrap
import sqlite3
import logging

import spacy
import nltk

from tqdm.auto import tqdm
# register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
tqdm.pandas()

# pandas display options
# https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html#available-options
pd.options.display.max_columns = 30 # default 20
pd.options.display.max_rows = 60 # default 60
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.precision = 2
pd.options.display.max_colwidth = 200 # default 50; -1 = all
# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

# np.set_printoptions(edgeitems=3) # default 3

import matplotlib
from matplotlib import pyplot as plt

plot_params = {'figure.figsize': (8, 4), 
               'axes.labelsize': 'large',
               'axes.titlesize': 'large',
               'xtick.labelsize': 'large',
               'ytick.labelsize':'large',
               'figure.dpi': 100}
# adjust matplotlib defaults
matplotlib.rcParams.update(plot_params)

import seaborn as sns
sns.set_style("darkgrid")

In [3]:
# this i modified to indicate where the document is stored
# on my laptop, so its directory is probably not going to 
# work for you, just change hte base dir location and should
# work

file = "eclipse_jdt.csv"
BASE_DIR = "/home/joe0400/Documents"
file = f"{BASE_DIR}/eclipse_jdt.csv.gz" ### real location
df = pd.read_csv(file)
print (df.columns)
df[['Issue_id','Priority','Component','Title','Description']].sample(2, random_state=42)



Index(['Issue_id', 'Priority', 'Component', 'Duplicated_issue', 'Title',
       'Description', 'Status', 'Resolution', 'Version', 'Created_time',
       'Resolved_time'],
      dtype='object')


,Issue_id,Priority,Component,Title,Description
38438,239715,P3,UI,No property tester for TestCaseElement for property projectNature,I20080613-2000; ; Not sure if this belongs to JDT/Debug or Platform/Debug.; ; I saw this error message several times today in my error log but Im not yet sure how to reproduce it.; ; -- Error Deta...
44129,395007,P3,UI,[package explorer] Refresh action not available on Java package folders,M3.; ; F5 (Refresh) is available as a context menu entry for ordinary source folders but not for Java package folders in the e4 Java Package explorer.; ; Please restore the 3.x functionality.


In [4]:
df = df[['Title','Description','Priority']] ### Remove other attributes
df = df.dropna()

In [5]:
distributions_of_df_priority_label = {key:sum([int(v == key) for v in df['Priority']]) for key in set(df['Priority'])}
probability_of_df_priority_label = {key:distributions_of_df_priority_label[key] / sum(distributions_of_df_priority_label.values()) for key in distributions_of_df_priority_label.keys()}
probability_of_df_priority_label

{'P5': 0.0055686915784588864,
 'P3': 0.8766932579055532,
 'P2': 0.06768612025722051,
 'P1': 0.024793936313614567,
 'P4': 0.025257993945152806}

In [6]:
df['text'] = df['Title'] + ' ' + df['Description'] ### Combine title and description into text
df = df.drop(columns=['Title','Description']) ### Drop title and description columns
df.columns

Index(['Priority', 'text'], dtype='object')

In [7]:
def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text) 
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df['text'] = df['text'].apply(clean)
df = df[df['text'].str.len() > 50]
df[0:2]

,Priority,text
0,P1,Icons needed for actions (1GI5UXW) JGS (8/8/01 5:20:19 PM); We need enabled; disabled hover icons for the following actions:; CopyToClipboardActionDelegate; RelaunchActionDelegate; TerminateAndRem...
1,P3,README: Hit count not reset (1GET20Y) JGS (6/4/01 11:43:47 AM); Set a breakpoint in InfiniteLoop with a hit count. When suspended;; right click breakpoint in Breakpoints viewer and enable. Hit cou...


In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(df['text'],
                                                    df['Priority'],
                                                    test_size=0.2, ### 80-20 train-test split
                                                    random_state=42,
                                                    stratify=df['Priority'])

print('Size of Training Data ', X_train.shape[0])
print('Size of Test Data ', X_test.shape[0])

Size of Training Data  36040
Size of Test Data  9011


In [33]:
# alright this creats a comparison dict between y_train and the priority labels, where each is stored as a probability

comparison = {v:sum([int(i == v) for i in Y_train.values]) / len(Y_train.values) for v in set(list(Y_train.values))}

In [34]:
comparison

{'P5': 0.00560488346281909,
 'P3': 0.8769145394006659,
 'P2': 0.06742508324084351,
 'P1': 0.024805771365149835,
 'P4': 0.025249722530521643}

In [35]:
from math import log
# this calculates kl_divergence using the formula provided
def kl_divergence(p,q):
    return sum((P * log(P/Q) for P,Q in zip(p,q)))
names = ['P1','P2','P3','P4','P5']
# this then uses the function to calculate by converting the dicts to aligned lists
print(f"kl divergince between training data and df = {kl_divergence([probability_of_df_priority_label[k] for k in names],[comparison[k] for k in names])}")

kl divergince between training data and df = 6.538546114509989e-07


In [36]:
# Sample 4000 bug reports with priority P3 
df_P3 = df[df['Priority'] == 'P3'].sample(n=4000, random_state=123)

# Create a separate dataframe containing all other bug reports
df_Rest = df[df['Priority'] != 'P3']

# Concatenate the two dataframes to create the new balanced bug reports dataset
df_balanced = pd.concat([df_Rest, df_P3])

# Check the status of the class imbalance


# this calculates the dictionary of counts for df_balanced
balanced_dictionary = {val: len([v for v in df_balanced['Priority'].values if v == val]) for val in set(df_balanced['Priority'].values)}

In [37]:
balanced_dictionary

{'P5': 252, 'P3': 4000, 'P2': 3038, 'P1': 1117, 'P4': 1138}

In [38]:
# this then devides it by its sum
balanced_probabilitys = {key:balanced_dictionary[key]/sum(balanced_dictionary.values()) for key in balanced_dictionary.keys()}

In [39]:
balanced_probabilitys

{'P5': 0.02640125720272394,
 'P3': 0.41906757464641176,
 'P2': 0.3182818229439497,
 'P1': 0.11702462022001048,
 'P4': 0.11922472498690413}

In [40]:

df_new = df_balanced[['text', 'Priority']]

In [41]:
# same for df_new
df_new_balance = {key:len([v for v in df_new['Priority'].values if v == key]) for key in set(df_new['Priority'].values)}

In [42]:
# on both of these
df_new_probabilitys = {key: df_new_balance[key]/sum(df_new_balance.values()) for key in df_new_balance.keys()}

In [43]:
df_new_probabilitys

{'P5': 0.02640125720272394,
 'P3': 0.41906757464641176,
 'P2': 0.3182818229439497,
 'P1': 0.11702462022001048,
 'P4': 0.11922472498690413}

In [44]:
# this then calculates the divergence and writes it to screen

In [45]:
print(f"kl divergence between df new and balanced is {kl_divergence([balanced_probabilitys[k] for k in names], [df_new_probabilitys[k] for k in names])}")

kl divergence between df new and balanced is 0.0


both balanced and df new are pretty much the same as the kl divergence is 0, and that indicates its the same distribution

however with y train and df it indicates there is some difference, but the difference is fairly minimal indicate it is different dataset, however they are fairly similar to each other.